## Load deep lab cut data

In [ ]:
import os
import numpy as np
from one.api import ONE
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import brainbox.behavior.wheel as wh
from scipy.stats import zscore
import concurrent.futures

# one = ONE(base_url='https://alyx.internationalbrainlab.org')
one = ONE(mode='remote')
# Get my functions
from video_functions_jaime import *

### Get QC of all sessions

In [ ]:
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Brainwidemap/'

os.chdir(data_path)
qc = pd.read_csv("Brainwidemap_QC - behavior_data.csv")


### Select sessions that pass QC for left DLC and left video

Video and DLC are set to pass if all tests in the respective categories pass as well

In [ ]:
pass_qc = qc.loc[(qc['leftVideo']=='PASS') & (qc['leftDLC']=='PASS'), 'Session ID']
print(str(len(pass_qc)) + ' sessions pass QC')

25 sessions pass QC


Some sessions pass leftDLC but not leftVideo. Why would this be?

In [ ]:
pass_qc = qc.loc[(qc['leftDLC']=='PASS'), 'Session ID']
print(str(len(pass_qc)) + ' sessions pass QC')

176 sessions pass QC


### Video that is both good on the left and right sides

In [ ]:
pass_qc = qc.loc[(qc['leftVideo']=='PASS') & (qc['leftDLC']=='PASS') &
                 (qc['rightVideo']=='PASS') & (qc['rightDLC']=='PASS'), 'Session ID']
print(str(len(pass_qc)) + ' sessions pass QC')

6 sessions pass QC


Look into the extended QC of a passed session

In [ ]:
example_session = list(pass_qc)[4]
extended_qc = one.get_details(example_session, True)['extended_qc']
extended_qc


{'task': 'WARNING',
 'dlcBody': 'PASS',
 'dlcLeft': 'PASS',
 'behavior': 1,
 'dlcRight': 'FAIL',
 'videoBody': 'WARNING',
 'videoLeft': 'PASS',
 'videoRight': 'PASS',
 '_task_iti_delays': 0.1706896551724138,
 '_videoBody_focus': True,
 '_videoLeft_focus': True,
 '_videoRight_focus': True,
 '_task_goCue_delays': 0.9965576592082617,
 '_task_trial_length': 0.9965576592082617,
 '_task_stimOn_delays': 0.9931153184165232,
 '_videoBody_position': True,
 '_videoLeft_position': True,
 '_dlcBody_paw_far_nan': None,
 '_dlcLeft_paw_far_nan': True,
 '_task_n_trial_events': 0.9913941480206541,
 '_task_reward_volumes': 1.0,
 '_task_stimOff_delays': 0.9931153184165232,
 '_videoBody_framerate': [True, 30.063],
 '_videoBody_pin_state': [True, 0, 0],
 '_videoLeft_framerate': [True, 60.006],
 '_videoLeft_pin_state': [True, 0, 0],
 '_videoRight_position': True,
 '_dlcBody_mean_in_bbox': True,
 '_dlcLeft_mean_in_bbox': True,
 '_dlcRight_paw_far_nan': False,
 '_task_audio_pre_trial': 1.0,
 '_task_errorCue_de

### Parameters

In [ ]:
bin_size = 0.1  # seconds
video_type = 'left'    

In [ ]:
# Can further filter based on actual frame rate (if it is not the expected one)
def check_fr(one, pass_qc, video_type):
    frame_rate = pass_qc.copy()
    frame_rate['frame_rate'] = np.zeros(len(pass_qc)) * np.nan
    for s, session in enumerate(pass_qc['session_uuid']):
        try:
            
            mouse_name = pass_qc['subject_nickname'][s]
            dlc_times, XYs = get_dlc_XYs(one, session, view=video_type, likelihood_thresh=0.9)
            frt = 1/np.mean(np.diff(dlc_times))
            
            # Save
            frame_rate.loc[(frame_rate['subject_nickname']==mouse_name) & 
                        (frame_rate['session_uuid']==session), 'frame_rate'] = frt

        except:
            print(session)
    return frame_rate

frame_rate = check_fr(pass_qc)

d32876dd-8303-4720-8e7e-20678dc2fd71


: 

: 

In [ ]:
for s, session in enumerate(pass_qc['session_uuid']):
    try:
        mouse_name = list(pass_qc.loc[pass_qc['session_uuid']==session, 'subject_nickname'])[0]

        session_trials = one.load_object(session, obj='trials', namespace='ibl')
        session_trials = session_trials.to_df()

        # Licks
        session_licks = get_lick_times(one, session, combine=False, video_type=video_type)

        # Wheel
        wheel = one.load_object(session, 'wheel', collection='alf')
        pos, wheel_times = wh.interpolate_position(wheel.timestamps, wheel.position)

        # # Pupil diameter
        pupil_dia_raw, pupil_dia_smooth, _, _ = (
            get_raw_and_smooth_position(one, session, video_type, ephys=True, position_function=get_pupil_diameter))
        dlc_times, XYs = get_dlc_XYs(one, session, view=video_type, likelihood_thresh=0.9)
        
        # # Pupil position
        pupil_X, pupil_Y = pupil_center(XYs)
        
        # Motion energy of whisker pad
        me_times, motion_energy = get_ME(one, session, video_type)       
        motion_energy = zscore(motion_energy, nan_policy='omit') 
        
        # Nose positions
        nose_X, nose_Y = nose_tip(XYs)

        # Left paw positions
        left_paw_X, left_paw_Y = left_paw(XYs)
        
        # Right paw positions
        right_paw_X, right_paw_Y = right_paw(XYs)
        
        # Code below is for further processing video data into design matrix (binning, merging, etc)
        """
        # Get session values
        # session_length = list(session_trials['stimOff_times'][-1:])[0]  # TODO: this might not work if stimOff times are missing
        # n_bins = int(np.floor(session_length/bin_size))
        # onsets = session_trials['stimOn_times']

        # Initialize dataframe
        licking = lick_rate(bin_size, session_licks, session_trials)
        wheel_vel = wheel_velocity(bin_size, wheel_times, pos, session_trials)
        wheel_disp = wheel_displacement(wheel_times, pos, session_trials, bin_size, onset_subtraction=True)
        pupil_dia = pupil_diam(dlc_times, pupil_dia_smooth, session_trials, bin_size, onset_subtraction=True)
        whisker_me = cont_bin(me_times, motion_energy, session_trials, bin_size)
        left_X = cont_bin(dlc_times, left_paw_X, session_trials, bin_size)
        left_Y = cont_bin(dlc_times, left_paw_Y, session_trials, bin_size)
        right_X = cont_bin(dlc_times, right_paw_X, session_trials, bin_size)
        right_Y = cont_bin(dlc_times, right_paw_Y, session_trials, bin_size)
        pupil_X_pos = cont_bin(dlc_times, pupil_X, session_trials, bin_size)
        pupil_Y_pos = cont_bin(dlc_times, pupil_Y, session_trials, bin_size)
        nose_X_pos = cont_bin(dlc_times, nose_X, session_trials, bin_size)
        nose_Y_pos = cont_bin(dlc_times, nose_Y, session_trials, bin_size)
        
        # Rename column names
        pupil_dia = pupil_dia.rename(columns={'pupil_final':'pupil_diameter'})
        whisker_me = whisker_me.rename(columns={'Values':'whisker_me'})
        left_X = left_X.rename(columns={'Values':'left_X'})
        left_Y = left_Y.rename(columns={'Values':'left_Y'})
        right_X = right_X.rename(columns={'Values':'right_X'})
        right_Y = right_Y.rename(columns={'Values':'right_Y'})
        pupil_X_pos = pupil_X_pos.rename(columns={'Values':'pupil_X'})
        pupil_Y_pos = pupil_Y_pos.rename(columns={'Values':'pupil_Y'})   
        nose_X_pos = nose_X_pos.rename(columns={'Values':'nose_X'})
        nose_Y_pos = nose_Y_pos.rename(columns={'Values':'nose_Y'})     
        
        # Merge all tables
        all_metrics = wheel_vel[['Bin', 'avg_wheel_vel']].dropna().merge(licking[['Bin', 'Lick count']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(wheel_disp[['Bin', 'wheel_final']], on='Bin', how='outer')
        # all_metrics = all_metrics.merge(pupil_dia[['Bin', 'pupil_diameter']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(whisker_me[['Bin', 'whisker_me']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_X[['Bin', 'left_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_Y[['Bin', 'left_Y']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(right_X[['Bin', 'right_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(right_Y[['Bin', 'right_Y']], on='Bin', how='outer')
        # all_metrics = all_metrics.merge(pupil_X_pos[['Bin', 'pupil_X']], on='Bin', how='outer')
        # all_metrics = all_metrics.merge(pupil_Y_pos[['Bin', 'pupil_Y']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(nose_X_pos[['Bin', 'nose_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(nose_Y_pos[['Bin', 'nose_Y']], on='Bin', how='outer')

        # Remove Nans before adding more data or else this breaks (not sure if this is really tru, but worked like this)
        all_metrics['Lick count'] = all_metrics['Lick count'].replace(np.NaN, 0)
        all_metrics = all_metrics.dropna().drop_duplicates()

        # # Save design matrix
        # data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v4_5Jul2024/' + str(bin_size) + '/'
        # os.chdir(data_path)
        # pickle.dump(all_metrics, open("design_matrix_" + str(session) + '_'  + mouse_name + '_' + str(bin_size), "wb"))
        """
    except:
       print(session)
